# 1. annotate vcf file

the location of data on genomDK is
/home/joco/faststorage/soyadapt_data_analysis_with_start_data/resources/start_data


i will work with SNPeff on genomedk
for this i will make a seperate projhect folder as the otherfolders are backups of the rest of my project from my laptop+github 

snpeff/
   README.txt
   resources/
       README.txt
       ...
   steps/
       README.txt
       ...
   results/
       README.txt
       ...
   plots/
       README.txt
       ...
   workflow/
       README.txt
       notebooks
       envs
         environment.yml


conda activate

conda create -n snpeffenv 

Download latest version
wget https://snpeff.blob.core.windows.net/versions/snpEff_latest_core.zip

Unzip file
unzip snpEff_latest_core.zip

i had to update to java 11 on the environment only



In [ ]:
#conda install -c conda-forge openjdk=11
java -version

#local !!! You have to be in the unzipped /snpEff file to run the following command! !!

#on genome dk go to 
cd /home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/
#found its location after downlading via conda with
# find / -name snpEff.jar 2>/dev/null

java -jar snpEff.jar databases | grep -i glycine_max

Glycine_max                                                 	Glycine_max                                                 	          	                              	[https://snpeff.blob.core.windows.net/databases/v5_1/snpEff_v5_1_Glycine_max.zip, https://snpeff.blob.core.windows.net/databases/v5_0/snpEff_v5_0_Glycine_max.zip]

 java -jar snpEff.jar download Glycine_max

collect the files

1. path to reference genome file (fasta) - the same as used for VCF construction

2. path to Reference genome gene annotation (gff) :

3. path to output directory

4. path to vcf input
5. the downloaded database data



in genomeDK:
1. ref=/home/joco/faststorage/snpeff_analysis/resources/Wm82.a2.v1.genome.fasta
2. ann=/home/joco/faststorage/snpeff_analysis/resources/Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz

3. out=/home/joco/faststorage/snpeff_analysis/resources/generated_data/
4. wgs_maf5=/home/joco/faststorage/snpeff_analysis/resources/start_data/wgs_maf5.vcf.gz
5. snpeff_data=/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/Glycine_max


i checked to see if the chromsomes have the same name in my vcf file as the .fasta and .gff3



zcat Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz | cut -f1 | sort | uniq

fasta file

zcat your_file.fasta.gz | grep "^>" | cut -d " " -f 1 | sed 's/>//'

so maybe not the same 



# Annotate snpeff with database

In [5]:
in:
conda activate snpeffenv
/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/

export wgs_maf5=/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/wgs_maf5.vcf.gz

java -Xmx8g -jar snpEff.jar -v Glycine_max  $wgs_maf5 > wgs_maf5.ann.vcf
bgzip wgs_maf5.ann.vcf
bcftools index wgs_maf5.ann.vcf.gz
bcftools query -l wgs_maf5.ann.vcf.gz | sort -u | wc -l
bcftools query -f '%CHROM\n' wgs_maf5.ann.vcf.gz | sort -u

####### im having a problem here. only 17 chromosomes included???
1
10
11
12
13
14
15
16
17
2
3
4
5
6
7
8
9

# i will make one more with stats!
java -Xmx8g -jar snpEff.jar -v -stats ex1.html Glycine_max $wgs_maf5 > 2_wgs_maf5.ann.vcf
bcftools query -l 2_wgs_maf5.ann.vcf.gz | sort -u | wc -l
bgzip 2_wgs_maf5.ann.vcf
bcftools index 2_wgs_maf5.ann.vcf.gz
bcftools query -l 2_wgs_maf5.ann.vcf.gz | sort -u | wc -l
#652
#bcftools query -f '%CHROM\n' wgs_maf5.vcf.gz | sort -u
bcftools query -f '%CHROM\t%POS\t%INFO/ANN\n' 2_wgs_maf5.ann.vcf.gz | \
awk -F '\t' 'BEGIN {OFS="\t"} { split($3, annFields, "|"); print $1, $2, annFields[3], annFields[4], annFields[5], annFields[6], annFields[7] }' > 2_extracted_genes.txt







SyntaxError: invalid syntax (841213456.py, line 1)

In [ ]:
#great i now have a file:
/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/wgs_maf5.ann.vcf.gz



i now want to compare the windows with differing values of signatures of selection in the list of genes against the rest of the genome

i have obtained a gene list from soybase and filtered for flowering or cold tolerence genes

In [ ]:
#I have the genes in a list one per line
genes_cold_soybase
genes_flowering_soybase
#match? with snpeff


so i need the Gene Coordinates: Obtain the genomic coordinates (start and end positions) of each gene in your gene list. 

In [ ]:
****
#So i go to the genomedk 
turn on my conda env

/home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/


In [ ]:
java -jar snpEff.jar -classic -onlyProtein genes_cold_soybase.txt wgs_maf5.ann.vcf.gz > gene_locations.txt

#not working

java -jar snpEff.jar filter "(ANN[*].GENEID = file('genes_cold_soybase.txt'))" wgs_maf5.ann.vcf.gz > gene_filtered.vcf


Calculate Average Signature Value: For each gene in your list, calculate the average value of signatures of selection across the windows that overlap with the gene's coordinates. You can write a script or use bioinformatics tools to do this.

Calculate Average Value for Rest of Genome: Calculate the average value of signatures of selection for all the windows in the rest of the genome (i.e., excluding the windows that overlap with the genes in your list).

Statistical Comparison: Perform a statistical comparison, such as a t-test or Wilcoxon rank-sum test, to determine if the average signature value in your gene list is significantly different from the average value in the rest of the genome.

Multiple Testing Correction (Optional): If you are comparing against a large number of windows, consider applying multiple testing correction methods, such as Bonferroni or False Discovery Rate (FDR) correction, to account for the increased risk of false positives.

Visualization: You can create visualizations to compare the average signature values between your gene list and the rest of the genome. Box plots, violin plots, or bar plots can be useful for this purpose.

bcftools view -h wgs_maf5.ann.vcf.gz   # View the VCF header
#checked out the annotation format
bcftools view wgs_maf5.ann.vcf.gz | less


i have a list of genes to try out


In [ ]:
# something java -jar snpEff.jar  floweringgenes > flowering_gene_positions.txt



In [ ]:
bcftools query -f '%CHROM\t%POS\t%INFO/ANN\n' wgs_maf5.ann.vcf.gz > output.txt
#im looking at ANN
bcftools query -f '%CHROM\t%POS\t%INFO/ANN\n' wgs_maf5.ann.vcf.gz | \
awk -F '\t' 'BEGIN {OFS="\t"} { split($3, annFields, "|"); print $1, $2, annFields[3], annFields[4], annFields[5], annFields[6], annFields[7] }' > output.txt

# Each annotation in the ANN field includes several subfields, which are separated by vertical bars (|). The exact subfields and their order can vary based on the version of SnpEff and the reference genome used, but they generally include information like:

    Allele: The alternative allele associated with the variant.
    Annotation: The functional consequence of the variant, such as "missense_variant," "frameshift_variant," "splice_acceptor_variant," etc.
    Annotation Impact: The impact level of the variant, such as "HIGH," "MODERATE," or "LOW," indicating the severity of the effect.
    Gene Name: The name of the gene affected by the variant.
    Gene ID: A unique identifier for the gene.
    Feature Type: The type of feature affected by the variant, such as "transcript," "exon," etc.
    Feature ID: A unique identifier for the feature.
    Transcript BioType: The biotype of the transcript.
    Rank / Total: The exon rank of the variant within the transcript, followed by the total number of exons.
    HGVS cDNA: The variant's description at the cDNA level.
    HGVS protein: The variant's description at the protein level.
    cDNA Position / CDS Position / Protein Position: The positions of the variant within the cDNA, CDS (coding sequence), and protein sequences, respectively.

##SnpEffVersion="5.1 (build 2022-01-21 06:23), by Pablo Cingolani"
##SnpEffCmd="SnpEff  Glycine_max /home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/wgs_maf5.vcf.gz "
##INFO=<ID=ANN,Number=.,Type=String,Description="Functional annotations: 'Allele | Annotation | Annotation_Impact | Gene_Name | Gene_ID | Feature_Type | Feature_ID | Transcript_BioType | Rank | HGVS.c | HGVS.p | cDNA.pos / cDNA.length | CDS.pos / CDS.length | AA.pos / AA.length | Distance | ERRORS / WARNINGS / INFO'">
##INFO=<ID=LOF,Number=.,Type=String,Description="Predicted loss of function effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">
##INFO=<ID=NMD,Number=.,Type=String,Description="Predicted nonsense mediated decay effects for this variant. Format: 'Gene_Name | Gene_ID | Number_of_transcripts_in_gene | Percent_of_transcripts_affected'">

1	27	G|intergenic_region|MODIFIER|CHR_START-GLYMA_01G000100|CHR_START-GLYMA_01G000100|intergenic_region|CHR_START-GLYMA_01G000100|||n.27A>G||||||
so $4


In [ ]:
##not working below

In [ ]:
#i moved loval to 
/Users/josephinelovemachine/soyadapt_data_analysis_with_start_data/resources/generated_data

In [ ]:
 sed 's/|/\t/g' output.txt | \
awk -F '\t' 'BEGIN{OFS=FS}{gsub("-", " ", $4); gsub("-", " ", $5); gsub("-", " ", $6); gsub("-", " ", $7); print $1, $2, $4, $5, $6, $7}' | \
awk -F '\t' '{print $1, $2, tolower($3), tolower($4), tolower($5), tolower($6)}' > extracted_genes.txt

In [1]:
java -jar snpEff.jar -classic -onlyProtein gene_list.txt input.vcf > gene_locations.txt


notes
scans for quality
stats?
https://github.com/alexherbig/MultiVCFAnalyzer#usage

grep -Ff extracted_genes.txt genes_cold_soybase.txt > 3.txt


# try another way by just extracting from the gff file?

@Genome Dk /home/joco/miniconda3/pkgs/snpeff-5.1-hdfd78af_0/share/snpeff-5.1-0/data/
Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz
Conda activate snpeffenv
bgzip -d Glycine_max.Glycine_max_v2.1.56.chr.gff3.gz

cut -f 3 Glycine_max.Glycine_max_v2.1.56.chr.gff3 | sort -u

In [ ]:
###
biological_region
CDS
chromosome
exon
five_prime_UTR
gene
#!genebuild-last-updated 2018-08
#!genome-build-accession GCA_000004515.4
#!genome-build Joint Genome Institute Glycine_max_v2.1
#!genome-date 2018-07
#!genome-version Glycine_max_v2.1
##gff-version 3
lnc_RNA
mRNA
ncRNA
ncRNA_gene
pre_miRNA
RNase_MRP_RNA
rRNA
##sequence-region   10 1 51566898
##sequence-region   11 1 34766867
##sequence-region   1 1 56831624
##sequence-region   12 1 40091314
##sequence-region   13 1 45874162
##sequence-region   14 1 49042192
##sequence-region   15 1 51756343
##sequence-region   16 1 37887014
##sequence-region   17 1 41641366
##sequence-region   18 1 58018742
##sequence-region   19 1 50746916
##sequence-region   20 1 47904181
##sequence-region   2 1 48577505
##sequence-region   3 1 45779781
##sequence-region   4 1 52389146
##sequence-region   5 1 42234498
##sequence-region   6 1 51416486
##sequence-region   7 1 44630646
##sequence-region   8 1 47837940
##sequence-region   9 1 50189764
snoRNA
snRNA
SRP_RNA
three_prime_UTR
tRNA

In [9]:
cut -f 3 Glycine_max.Glycine_max_v2.1.56.chr.gff3 | grep -E 'biological_region|CDS|chromosome|exon|gene' | sort -u
# extract

awk -F '\t' '$3 ~ /^(chromosome|gene)$/ { print $1, $4, $9 }' Glycine_max.Glycine_max_v2.1.56.chr.gff3 > extracted_info.txt
# looks like this:
#1 90289 ID=gene:GLYMA_01G000500;biotype=protein_coding;description=hypothetical protein;gene_id=GLYMA_01G000500
#and want this:
#1 90289 gene_id=GLYMA_01G000500
#the this
#1 90289 glyma_01g000500
awk '{ match($0, /([0-9]+)\s+([0-9]+).*gene_id=([^;]+)/, arr); print arr[1], arr[2], arr[3] }' extracted_info.txt > extracted_info1.txt
awk '{ match($0, /([0-9]+)\s+([0-9]+).*gene_id=([^;]+)/, arr); print arr[1], arr[2], tolower(arr[3]) }' extracted_info.txt > extracted_gff3.txt
#moved to home dir

SyntaxError: invalid syntax (311688914.py, line 1)